In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', 70)

os.getcwd()

'/Users/miek/Documents/DS4B/DS4BThesisSNCB'

In [2]:
traject = pd.read_excel('./data/Traject per P train.xlsx')
traject

,Date,KAR,Train,Type,Traject
0,2019-01-03,R4,7001,P,OOSTENDE - SCHAARBEEK
1,2019-04-26,R5,7001,P,OOSTENDE - SCHAARBEEK
2,2019-06-21,R5,7001,P,OOSTENDE - SCHAARBEEK
3,2020-01-21,R2,7001,P,OOSTENDE - SCHAARBEEK
4,2020-02-04,R2,7001,P,OOSTENDE - SCHAARBEEK
...,...,...,...,...,...
346152,2021-04-28,R3,8999,P,GENT-ST-P - KORTRIJK
346153,2022-04-13,R3,8999,P,GENT-ST-P - KORTRIJK
346154,2022-06-14,R2,8999,P,GENT-ST-P - KORTRIJK
346155,2022-09-20,R2,8999,P,GENT-ST-P - KORTRIJK


In [4]:
stops_post_covid = pd.read_parquet("~/Downloads/all_stops_ungrouped/all_stops_post_covid.parquet")
df = stops_post_covid
PTCAR = pd.read_csv("~/Documents/DS4B/DS4BThesisSNCB/data/PTCAR.csv",sep=";")
df = df.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
#df['p_resp'] = df['TRAIN_NUMBER'].apply(lambda x: 1 if ((x >= 7000) & (x <= 8999)) else 0)
#df = df[df['p_resp'] == 1]
df

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR,YEAR_DEP,month,season,weekend,holiday,dep_hour_origin,day_origin,date_origin,start_lifetime,end_lifetime,arr_hour_terminus,p_indicator,SYMBOLIC_NAME,GPS_LATITUDE,GPS_LONGITUDE,LONG_NAME_FR,LONG_NAME_NL,SHORT_NAME_FR,SHORT_NAME_NL,UIC_CODE,CLASSIFICATION
0,30539534,5329,2022-06-23,321,8,2022-06-23 08:09:00,2022-06-23 08:09:00,2022-06-23 08:11:53,2022-06-23 08:13:15,173,255,2,0,NaN,None,2022,6.0,Summer,0.0,0.0,7.266667,23.0,2022-06-23,2022-06-23 07:16:00,2022-06-23 09:55:00,9.916667,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30431291,9240,2022-06-05,1064,2,2022-06-05 13:41:00,2022-06-05 13:43:00,2022-06-05 13:44:02,2022-06-05 13:46:01,182,181,2,0,NaN,None,2022,6.0,Summer,1.0,1.0,13.466667,5.0,2022-06-05,2022-06-05 13:28:00,2022-06-05 16:23:00,16.383333,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30431291,9240,2022-06-05,1026,3,2022-06-05 14:09:00,2022-06-05 14:11:00,2022-06-05 14:12:10,2022-06-05 14:14:52,190,232,2,0,NaN,None,2022,6.0,Summer,1.0,1.0,13.466667,5.0,2022-06-05,2022-06-05 13:28:00,2022-06-05 16:23:00,16.383333,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30431291,9240,2022-06-05,1838,4,2022-06-05 14:34:00,2022-06-05 14:42:00,2022-06-05 14:36:38,2022-06-05 14:43:02,158,62,2,0,NaN,None,2022,6.0,Summer,1.0,1.0,13.466667,5.0,2022-06-05,2022-06-05 13:28:00,2022-06-05 16:23:00,16.383333,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30431291,9240,2022-06-05,88,1,1900-01-01 00:00:00,2022-06-05 13:28:00,1900-01-01 00:00:00,2022-06-05 13:30:20,0,140,1,0,NaN,None,2022,6.0,Summer,1.0,1.0,13.466667,5.0,2022-06-05,2022-06-05 13:28:00,2022-06-05 16:23:00,16.383333,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12910234,31329661,2627,2022-10-28,715,117,2022-10-28 06:13:00,2022-10-28 06:24:00,2022-10-28 06:13:49,2022-10-28 06:24:16,49,16,2,276,53.0,53,2022,10.0,Fall,0.0,0.0,5.616667,28.0,2022-10-28,2022-10-28 05:37:00,2022-10-28 07:12:00,7.200000,0,FLV,50.881356,4.716096,LEUVEN,LEUVEN,LEUVEN,LEUVEN,33001.0,1.0
12910235,31325182,2131,2022-10-28,70,103,2022-10-28 08:31:00,2022-10-28 08:36:00,2022-10-28 08:28:29,2022-10-28 08:36:30,-150,30,2,532,162.0,162,2022,10.0,Fall,0.0,0.0,8.600000,28.0,2022-10-28,2022-10-28 08:36:00,2022-10-28 11:28:00,11.466667,0,LL,49.679975,5.810614,ARLON,ARLON,ARLON,ARLON,66001.0,1.0
12910236,31326770,5468,2022-10-28,726,106,2022-10-28 19:02:00,2022-10-28 19:06:00,2022-10-28 19:28:05,2022-10-28 19:29:49,1565,1429,2,259,37.0,37A,2022,10.0,Fall,0.0,0.0,18.650000,28.0,2022-10-28,2022-10-28 18:39:00,2022-10-28 19:23:00,19.383333,0,FL,50.625482,5.565207,LIEGE-GUILLEMINS,LIEGE-GUILLEMINS,LIEGE-GUILL,LIEGE-GUILL,41004.0,1.0
12910237,31330053,555,2022-10-28,455,115,2022-10-28 06:12:00,2022-10-28 06:19:00,2022-10-28 06:14:13,2022-10-28 06:19:58,133,58,2,647,50.0,50E,2022,10.0,Fall,0.0,0.0,5.516667,28.0,2022-10-28,2022-10-28 05:31:00,2022-10-28 07:28:00,7.466667,0,FGSP,51.035763,3.710232,GENT-SINT-PIETERS,GENT-SINT-PIETERS,GENT-ST-P,GENT-ST-P,92007.0,1.0


In [5]:
min_hr_dep = df.groupby('REF_TOD_TRAIN_ID')['PTCAR_ORD_NO'].min().reset_index()
df_dep = pd.merge(df[['REF_TOD_TRAIN_ID', 'PTCAR_ORD_NO', 'LONG_NAME_NL']], min_hr_dep, 
                  on=['REF_TOD_TRAIN_ID', 'PTCAR_ORD_NO'], how='right')
df_dep = df_dep.rename(columns={'LONG_NAME_NL':"starting_station"})
df_dep

,REF_TOD_TRAIN_ID,PTCAR_ORD_NO,starting_station
0,30282277,1,NaN
1,30282278,1,NaN
2,30282279,101,MOL
3,30282280,101,ANTWERPEN-CENTRAAL
4,30282281,101,KORTRIJK
...,...,...,...
1093855,32455649,106,BRUGGE
1093856,32456014,101,AALTER
1093857,32456196,101,GENT-SINT-PIETERS
1093858,32456359,101,ARLON


In [6]:
# Assuming the data is in a Pandas DataFrame called df
max_hr_arr = df.groupby('REF_TOD_TRAIN_ID')['PTCAR_ORD_NO'].max().reset_index()
df_end = pd.merge(df[['REF_TOD_TRAIN_ID', 'PTCAR_ORD_NO', 'LONG_NAME_NL']], max_hr_arr,
                  on=['REF_TOD_TRAIN_ID', 'PTCAR_ORD_NO'], how='right')
df_end = df_end.rename(columns={'LONG_NAME_NL': 'ending_station'})
df_end

,REF_TOD_TRAIN_ID,PTCAR_ORD_NO,ending_station
0,30282277,107,KORTRIJK
1,30282278,136,PUURS
2,30282279,124,ANTWERPEN-CENTRAAL
3,30282280,152,NIVELLES
4,30282281,172,WELKENRAEDT
...,...,...,...
1093855,32455649,117,GENT-SINT-PIETERS
1093856,32456014,107,BRUGGE
1093857,32456196,116,KOMEN
1093858,32456359,101,ARLON


In [7]:
start_end_station = df_dep.merge(df_end, on = 'REF_TOD_TRAIN_ID', how = 'inner')
start_end_station = start_end_station.drop(columns=['PTCAR_ORD_NO_x','PTCAR_ORD_NO_y'])
start_end_station

,REF_TOD_TRAIN_ID,starting_station,ending_station
0,30282277,NaN,KORTRIJK
1,30282278,NaN,PUURS
2,30282279,MOL,ANTWERPEN-CENTRAAL
3,30282280,ANTWERPEN-CENTRAAL,NIVELLES
4,30282281,KORTRIJK,WELKENRAEDT
...,...,...,...
1093855,32455649,BRUGGE,GENT-SINT-PIETERS
1093856,32456014,AALTER,BRUGGE
1093857,32456196,GENT-SINT-PIETERS,KOMEN
1093858,32456359,ARLON,ARLON


In [9]:
start_end_station.to_csv('./data/start_end_station.csv', index=False)